# Instalando o PySpark no Google Colab

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").getOrCreate()

# Spark SQL - Consultas e Seleções

In [3]:
df = spark.sql('''select 'OK' as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



# Importing Data

In [4]:
df = spark.read.csv('cereal.csv', sep = ',', inferSchema=True, header = True)
df.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [5]:
df.count()

77

# Manipulation Data with Spark SQL

In [6]:
df.createOrReplaceTempView("cereal")

In [7]:
cereal = spark.sql('''SELECT COUNT(*) AS total FROM cereal WHERE mfr = 'G' ''')
cereal.show()

+-----+
|total|
+-----+
|   22|
+-----+



In [8]:
df.where(df['mfr']=='G').count()

22

# Select Disctinct on SparkSQL

In [9]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [10]:
df.createOrReplaceTempView("cereal")

In [11]:
cereal = spark.sql('''SELECT  DISTINCT type, mfr FROM cereal''')
cereal.show()

+----+---+
|type|mfr|
+----+---+
|   C|  P|
|   C|  Q|
|   C|  N|
|   H|  Q|
|   C|  R|
|   H|  N|
|   C|  G|
|   H|  A|
|   C|  K|
+----+---+



# WHERE no Spark SQL

In [12]:
cereal = spark.sql('''SELECT * FROM cereal WHERE mfr = 'K' ''''')
cereal.count()

23

In [13]:
cereal = spark.sql('''SELECT * FROM cereal WHERE calories =100 ''''')
cereal.count()

17

In [14]:
cereal = spark.sql('''SELECT * FROM cereal WHERE mfr = 'K' AND calories =100 ''''')
cereal.count()

3

#GROUP BY

In [15]:
cereal = spark.sql('''SELECT mfr, \
                             type, \
                             count(*) AS total,
                             sum(calories) AS total_calories
                      FROM cereal
                      GROUP BY  \
                            mfr, \
                            type ''''')
cereal.show()

+---+----+-----+--------------+
|mfr|type|total|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  P|   C|    9|           980|
|  K|   C|   23|          2500|
|  G|   C|   22|          2450|
|  Q|   C|    7|           660|
|  R|   C|    8|           920|
|  Q|   H|    1|           100|
|  N|   H|    1|           100|
|  N|   C|    5|           420|
+---+----+-----+--------------+



# CASE WHEN

In [16]:
cereal = spark.sql(''' SELECT DISTINCT type
                        FROM cereal''')
cereal.show()

+----+
|type|
+----+
|   C|
|   H|
+----+



In [17]:
cereal = spark.sql(''' SELECT mfr,
                              type,
                             (case
                                    when type = 'C' then 'A'
                                    --when type = 'H' then 'B'
                                    else 'B'
                             end) as type_new,
                             count(*) AS total,
                             sum(calories) AS total_calories
                      FROM cereal
                      GROUP BY
                            mfr,
                            type ''''')
cereal.show()

+---+----+--------+-----+--------------+
|mfr|type|type_new|total|total_calories|
+---+----+--------+-----+--------------+
|  A|   H|       B|    1|           100|
|  P|   C|       A|    9|           980|
|  K|   C|       A|   23|          2500|
|  G|   C|       A|   22|          2450|
|  Q|   C|       A|    7|           660|
|  R|   C|       A|    8|           920|
|  Q|   H|       B|    1|           100|
|  N|   H|       B|    1|           100|
|  N|   C|       A|    5|           420|
+---+----+--------+-----+--------------+



# Consultas Avançadas em SQL usando PySpark

In [18]:
df.show(5)

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
+---------------

In [21]:
cereal = spark.sql('''
                  SELECT mfr,
                         type,
                         sum(calories) as sum_calories,
                         min(calories) as min_calories,
                         max(calories) as max_calories,
                         cast (avg(calories) as decimal(10,2)) as avg_calories,

                         sum(carbo) as sum_carbo,
                         min(carbo) as min_carbo,
                         max(carbo) as max_carbo,
                         cast (avg(carbo) as decimal(10,2)) as avg_carbo,

                         sum(vitamins) as sum_vitamins,
                         min(vitamins) as min_vitamins,
                         max(vitamins) as max_vitamins,
                         cast (avg(vitamins) as decimal(10,2)) as avg_vitamins,

                         count(distinct name) as count_distinct_names,
                         count(name) as count_names
                  FROM cereal
                  GROUP BY mfr, type
                  ORDER BY mfr, type ''')
cereal.show()

+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|   C|        2500|          50|         160|      108.7

In [24]:
cereal = spark.sql('''
                  SELECT mfr,
                         type,
                         (case
                              when mfr = 'A' then 'Abacaxi'
                              when mfr = 'G' then 'Goiaba'
                              when mfr = 'K' then 'Banana'
                              when mfr = 'N' then 'Maça'
                              when mfr = 'P' then 'Tomate'
                              when mfr = 'Q' then 'Pera'
                              --when mfr = 'R'then 'Uva'
                              else 'NA'
                         end) as type_fruit,

                         sum(calories) as sum_calories,
                         min(calories) as min_calories,
                         max(calories) as max_calories,
                         cast (avg(calories) as decimal(10,2)) as avg_calories,

                         sum(carbo) as sum_carbo,
                         min(carbo) as min_carbo,
                         max(carbo) as max_carbo,
                         cast (avg(carbo) as decimal(10,2)) as avg_carbo,

                         sum(vitamins) as sum_vitamins,
                         min(vitamins) as min_vitamins,
                         max(vitamins) as max_vitamins,
                         cast (avg(vitamins) as decimal(10,2)) as avg_vitamins,

                         count(distinct name) as count_distinct_names,
                         count(name) as count_names
                  FROM cereal
                  GROUP BY mfr, type
                  ORDER BY mfr, type ''')
cereal.show()

+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|type_fruit|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|   Abacaxi|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|    Goiaba|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|

# JOINs

# INNER

In [25]:
sales = spark.read.csv('sales_data_sample.csv', sep = ',', inferSchema=True, header = True)

In [33]:
sales.show(5)

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+-----+----------+-------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003 0:00|Shipped| 

In [28]:
sales.createOrReplaceTempView('sales')

In [39]:
calendar = spark.sql('''
                  SELECT DISTINCT orderdate, qtr_id, month_id, year_id
                  FROM sales
                  ORDER BY orderdate
                ''')

sales_data = spark.sql('''
                  SELECT DISTINCT ORDERNUMBER,
                                  CUSTOMERNAME,
                                  ORDERDATE,
                                  SALES,
                                  QUANTITYORDERED,
                                  PRODUCTCODE,
                                  ORDERLINENUMBER,
                                  PRICEEACH
                  FROM sales
                  ORDER BY ORDERNUMBER
                ''')

customers = spark.sql('''
                  SELECT DISTINCT CUSTOMERNAME,
                                  PHONE,
                                  ADDRESSLINE1,
                                  ADDRESSLINE2,
                                  CITY,
                                  STATE,
                                  POSTALCODE,
                                  COUNTRY,
                                  TERRITORY
                  FROM sales
                  ORDER BY CUSTOMERNAME
                ''')
sales_data.createOrReplaceTempView('sales_data')
calendar.createOrReplaceTempView('calendar')
customers.createOrReplaceTempView('customers')

In [40]:
calendar.count()

252

In [41]:
sales_data.count()

2823

In [42]:
customers.count()

92

In [51]:
master = spark.sql('''

                  SELECT DISTINCT s.ordernumber, c.city
                  FROM sales_data s
                  INNER JOIN customers c ON s.CUSTOMERNAME=c.CUSTOMERNAME

                  ''')
master.show()

+-----------+-------------+
|ordernumber|         city|
+-----------+-------------+
|      10300|    Frankfurt|
|      10385|   San Rafael|
|      10241|   Strasbourg|
|      10182|   San Rafael|
|      10140|   Burlingame|
|      10153|       Madrid|
|      10293|       Torino|
|      10161|      Aaarhus|
|      10406|    Kobenhavn|
|      10414|       Boston|
|      10311|       Madrid|
|      10357|   San Rafael|
|      10195| White Plains|
|      10189|     Pasadena|
|      10422|    Allentown|
|      10111|San Francisco|
|      10204|          NYC|
|      10151|         Oulu|
|      10304|   Versailles|
|      10369|   Brickhaven|
+-----------+-------------+
only showing top 20 rows

